# Setup

In [1]:
# Import stuff to be used in this notebook. Run this first!
%reload_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np
import meshcat
import meshcat.geometry as g
from manip_station_sim.plan_utils import PlotJointLog, PlotIiwaPositionLog

In [2]:
# First start meshcat for visualization - this only has to be run once.

# If you interrupt the kernel of this notebook, you'll need to run this cell again to 
# restart the meshcat server, and then refresh the visualization window. 

# This will open a mesh-cat server in the background, 
# click on the url to display visualization in a separate window. 
vis = meshcat.Visualizer()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


In [3]:
def make_meshcat_color_array(N, r, g, b):
    color = np.zeros((3, N))
    color[0, :] = r
    color[1, :] = g
    color[2, :] = b

    return color

In [4]:
box_model = np.load("resources/box_model.npy")


In [5]:
M = box_model.shape[0]
red = make_meshcat_color_array(M, 0.5, 0, 0)
green = make_meshcat_color_array(M, 0, 0.5, 0)
model_meshcat = g.PointCloud(box_model.T, red)
vis['model'].set_object(model_meshcat)

In [ ]:
# Run this if you feel the visualizer is messed up and want to start over.
vis.delete()

# Inverse Kinematics

In [6]:
from manip_station_sim.manipulation_station_simulator import ManipulationStationSimulator
from manip_station_sim.robot_plans import JointSpacePlan
from iiwa_pick_and_place_trajectory_generation import GenerateIiwaTrajectoriesAndGripperSetPoints
from pydrake.math import RollPitchYaw
from pydrake.math import RigidTransform

q0 = np.array([0., 0, 0., -1.5, 0., 1., 0.])

X_WO = RigidTransform()
X_WO.set_translation([0.555, 0.02, 0.1])

rpy = RollPitchYaw([np.pi/2,0, -0.106])
X_WO.set_rotation(rpy)

# Construct simulator.
sim = ManipulationStationSimulator(X_WO)

# Create Plans
q_traj_list, gripper_setpoint_list = GenerateIiwaTrajectoriesAndGripperSetPoints(X_WO)
plan_list = [JointSpacePlan(q_traj) for q_traj in q_traj_list]

# real_time_rate=0 means running the simulation as fast as possible. 
# Setting real_time_rate 1 to run the simulation close to real time.
(iiwa_position_command_log, iiwa_position_measured_log,
    iiwa_external_torque_log, iiwa_velocity_estimated_log,
    plant_state_log, t_plan) = sim.RunSimulation(
    q0, plan_list, gripper_setpoint_list, extra_time=2.0, real_time_rate=0.3)

SolutionResult.kSolutionFound
Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6000...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/
Connected to meshcat-server.
simulation duration 39.0
plan starting times
 [ 0.  2.  9. 12. 14. 18. 23. 28. 33. 37.]
t:  0.0
t:  1.0
Running plan 1 (type: JointSpacePlanGoToTarget), starting at 2.000000 for a duration of 6.000000 seconds.

t:  2.0
t:  3.0
t:  4.0
t:  5.0
t:  6.0
t:  7.0
t:  8.0
Running plan 2 (type: JointSpacePlan), starting at 9.000000 for a duration of 2.000000 seconds.

t:  9.0
t:  10.0
t:  11.0
Running plan 3 (type: JointSpacePlan), starting at 12.000000 for a duration of 1.000000 seconds.

t:  12.0
t:  13.0
Running plan 4 (type: JointSpacePlan), starting at 14.000000 for a duration of 3.000000 seconds.

t:  14.0
t:  15.0
t:  16.0
t:  17.0
Running plan 5 (type: JointSpacePlan), starting at 18.000000 for a duration of 4.000000 seconds.

t:  18.0
t:  19.0
t:  20.0
t:  21.0
t:  22.0
Runni

## Plots

In [ ]:
# Plot joint velocity
PlotJointLog(iiwa_velocity_estimated_log, legend="qDt", y_label="rad/s")

In [ ]:
# Plot joint torque
PlotJointLog(iiwa_external_torque_log, legend="tau_ext", y_label="N*m")

In [ ]:
# Plot joint position logs.
PlotIiwaPositionLog(iiwa_position_command_log, iiwa_position_measured_log)

## Utility

In [ ]:
# utility: draw a frame at pose X_WC.
from pydrake.systems.meshcat_visualizer import AddTriad
from pydrake.math import RigidTransform
X_WC = RigidTransform()
X_WC.set_translation([0.52, 0.033, 0.21]) 

rpy = RollPitchYaw([0,0, -0.106])
X_WC.set_rotation(rpy)


AddTriad(vis, "test", "tests", length=0.15, radius=0.006)
vis["tests"]["test"].set_transform(X_WC.matrix())

# RRT


In [7]:
# Import stuff to be used in this notebook. Run this first!
%reload_ext autoreload
%autoreload 2

import numpy as np
import meshcat

from pydrake.math import RollPitchYaw, RotationMatrix, RigidTransform
from pydrake.trajectories import PiecewisePolynomial

In [9]:
from ik_iiwa import SolveOneShotIk, q0, q_home, p_L7Q, X_WO


p_WQ_0 = np.array([0.52, 0.033, 0.18])
R_WL7_0 = RollPitchYaw(0.14, np.pi * 5 / 6, 0).ToRotationMatrix()

p_WQ_final = np.array([0.53, 0.033, 0.38])
R_WL7_final = RollPitchYaw(0.14, np.pi * 5 / 6, 0).ToRotationMatrix()

q_start = SolveOneShotIk(p_WQ_0, R_WL7_0, p_L7Q, q_home)
q_goal = SolveOneShotIk(p_WQ_final, R_WL7_final, p_L7Q, q_home)

In [10]:

p_WQ_0_2 = np.array([0.53, 0.033, 0.38])
R_WL7_0_2 = RollPitchYaw(0.14, np.pi * 5 / 6, 0).ToRotationMatrix()

p_WQ_final_2 = np.array([0.52, 0.033, 0.21])
R_WL7_final_2 = RollPitchYaw(0.14, np.pi * 5 / 6, 0).ToRotationMatrix()

q_start_2 = SolveOneShotIk(p_WQ_0_2, R_WL7_0_2, p_L7Q, q_home)
q_goal_2 = SolveOneShotIk(p_WQ_final_2, R_WL7_final_2, p_L7Q, q_home)

In [11]:
# Constructs rrt problem 1. 
from rrt_planner.iiwa_rrt_problem import IiwaProblem

gripper_setpoint = 0.1
door_angle = np.pi/2 - 0.001
left_door_angle = -0.001
right_door_angle = 0.001

iiwa_problem = IiwaProblem(
    q_start=q_start,
    q_goal=q_goal,
    gripper_setpoint=gripper_setpoint,
    left_door_angle=left_door_angle,
    right_door_angle=right_door_angle,
    is_visualizing=True)

Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6000...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/
Connected to meshcat-server.


In [12]:
# Constructs rrt problem 2. 
from rrt_planner.iiwa_rrt_problem import IiwaProblem

iiwa_problem_2 = IiwaProblem(
    q_start=q_start_2,
    q_goal=q_goal_2,
    gripper_setpoint=gripper_setpoint,
    left_door_angle=left_door_angle,
    right_door_angle=right_door_angle,
    is_visualizing=True)

Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6000...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/
Connected to meshcat-server.


In [13]:
path = iiwa_problem.run_planner(method="birrt")

Path found with 2 movements of distance  3.2078283481154464
Smoothed path found with 2 movements of distance  1.3313599284380206


In [14]:
path_2 = iiwa_problem_2.run_planner(method="birrt")

Path found with 2 movements of distance  6.328087305886326
Smoothed path found with 1 movements of distance  1.074291576235297


In [15]:
from manip_station_sim_rrt.manipulation_station_simulator import ManipulationStationSimulator
from manip_station_sim_rrt.robot_plans import JointSpacePlan

X_WO = RigidTransform()
X_WO.set_translation([0.555, 0.02, 0.1])

rpy = RollPitchYaw([np.pi/2,0, -0.106])
X_WO.set_rotation(rpy)


# joint space trajectory from RRT path
duration = 8
t_knots = np.linspace(0, duration, len(path))
q_knots = np.array(path)
q_traj_rrt = PiecewisePolynomial.Cubic(t_knots, q_knots.T, np.zeros(7), np.zeros(7))

#joint space trajectory from RRT path
duration_2 = 8
t_knots_2 = np.linspace(0, duration_2, len(path_2))
q_knots_2 = np.array(path_2)
q_traj_rrt_2 = PiecewisePolynomial.Cubic(t_knots_2, q_knots_2.T, np.zeros(7), np.zeros(7))

# close gripper
q_knots_close_gripper = np.zeros((2, 7))
q_knots_close_gripper[0] = q_knots[0]
qtraj_close_gripper = PiecewisePolynomial.ZeroOrderHold(
                            [0, 1], q_knots_close_gripper.T)

# open gripper
q_knots_open_gripper = np.zeros((2, 7))
q_knots_open_gripper[0] = q_knots_2[-1]
qtraj_open_gripper = PiecewisePolynomial.ZeroOrderHold(
                            [0, 1], q_knots_open_gripper.T)

# Generate plan list
plan_list = [JointSpacePlan(qtraj) for qtraj in [qtraj_close_gripper, q_traj_rrt, q_traj_rrt_2, qtraj_open_gripper]]
gripper_setpoint_list = [0.005, 0.005, 0.005, 0.1]


# Construct simulator.
sim = ManipulationStationSimulator(X_WO, robot_has_collision=True)

door_angles = np.array([left_door_angle, right_door_angle])
# real_time_rate=0 means running the simulation as fast as possible. 
# Setting real_time_rate 1 to run the simulation close to real time.
(iiwa_position_command_log, iiwa_position_measured_log,
    iiwa_external_torque_log, iiwa_velocity_estimated_log,
    plant_state_log, t_plan) = sim.RunSimulation(
    q_home, door_angles, plan_list, gripper_setpoint_list, extra_time=2.0, real_time_rate=0.0,
    gripper_setpoint_initial=0.1)


Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6000...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/
Connected to meshcat-server.
simulation duration 33.0
plan starting times
 [ 0.  2.  9. 11. 20. 29. 31.]
t:  0.0
t:  1.0
Running plan 1 (type: JointSpacePlanGoToTarget), starting at 2.000000 for a duration of 6.000000 seconds.

t:  2.0
t:  3.0
t:  4.0
t:  5.0
t:  6.0
t:  7.0
t:  8.0
Running plan 2 (type: JointSpacePlan), starting at 9.000000 for a duration of 1.000000 seconds.

t:  9.0
t:  10.0
Running plan 3 (type: JointSpacePlan), starting at 11.000000 for a duration of 8.000000 seconds.

t:  11.0
t:  12.0
t:  13.0
t:  14.0
t:  15.0
t:  16.0
t:  17.0
t:  18.0
t:  19.0
Running plan 4 (type: JointSpacePlan), starting at 20.000000 for a duration of 8.000000 seconds.

t:  20.0
t:  21.0
t:  22.0
t:  23.0
t:  24.0
t:  25.0
t:  26.0
t:  27.0
t:  28.0
Running plan 5 (type: JointSpacePlan), starting at 29.000000 for a duration of 1.000000 sec